In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel(
    r"source_data/Тестовое_задание_Data_аналитик_ЦО_2025_.xlsx",
    sheet_name="Данные для задачи 3"
)

df.head()

In [ ]:
# Переименовываем для удобства
data = df.rename(columns={
    "Бренд AX 1": "brand",
    "Неделя": "week",
    "Выручка": "revenue",
    "Маржинальность": "margin_rate",
})

# Считаем валовую прибыль (маржа в деньгах) = выручка * маржинальность в %
data["gp"] = data["revenue"] * data["margin_rate"]

data.head()

In [ ]:
# Сводная таблица по неделям
totals = data.groupby("week").agg(
    revenue=("revenue", "sum"),
    gp=("gp", "sum"),
)
totals["margin_rate"] = totals["gp"] / totals["revenue"]

w_prev2, w_prev = 14, 15

rev_14, rev_15 = totals.loc[w_prev2, "revenue"], totals.loc[w_prev, "revenue"]
gp_14, gp_15 = totals.loc[w_prev2, "gp"], totals.loc[w_prev, "gp"]
m_14, m_15 = totals.loc[w_prev2, "margin_rate"], totals.loc[w_prev, "margin_rate"]

# Расчитываем дельту
delta_rev = rev_15 - rev_14
delta_gp = gp_15 - gp_14
delta_m_pp = (m_15 - m_14) * 100

summary = pd.DataFrame({
    "Неделя 14": [f"{rev_14:,.0f}", f"{gp_14:,.0f}", f"{m_14*100:.2f}%"],
    "Неделя 15": [f"{rev_15:,.0f}", f"{gp_15:,.0f}", f"{m_15*100:.2f}%"],
    "Изменение": [f"{delta_rev:,.0f}", f"{delta_gp:,.0f}", f"{delta_m_pp:+.2f} п.п."],
}, index=["Выручка", "Валовая прибыль", "Маржинальность"])

summary

In [ ]:
# Создаём pivot по брендам
pivot = data.pivot_table(index="brand", columns="week", values=["revenue", "gp"], aggfunc="sum").fillna(0)

# Маржинальность по брендам
mr_14 = np.where(pivot[("revenue", w_prev2)] > 0, pivot[("gp", w_prev2)] / pivot[("revenue", w_prev2)], 0)
mr_15 = np.where(pivot[("revenue", w_prev)] > 0, pivot[("gp", w_prev)] / pivot[("revenue", w_prev)], 0)

# Конечная таблица по брендам
brands = pd.DataFrame({
    "rev_w14": pivot[("revenue", w_prev2)],
    "rev_w15": pivot[("revenue", w_prev)],
    "delta_rev": pivot[("revenue", w_prev)] - pivot[("revenue", w_prev2)],
    "margin_w14": mr_14,
    "margin_w15": mr_15,
    "delta_margin_pp": (mr_15 - mr_14) * 100,
    "gp_w14": pivot[("gp", w_prev2)],
    "gp_w15": pivot[("gp", w_prev)],
    "delta_gp": pivot[("gp", w_prev)] - pivot[("gp", w_prev2)],
}).sort_values("delta_gp", ascending=False)

# Топ 7 брендов по увеличению валовой прибыли
print("ТОП 7 БРЕНДОВ ПО РОСТУ ВАЛОВОЙ ПРИБЫЛИ (GP):")
print(brands[["delta_gp", "delta_rev", "margin_w14", "margin_w15"]].head(7).to_string())

print("\n\nТОП 7 БРЕНДОВ ПО ПАДЕНИЮ ВАЛОВОЙ ПРИБЫЛИ (GP):")
print(brands[["delta_gp", "delta_rev", "margin_w14", "margin_w15"]].tail(7).to_string())

In [ ]:
# Доля выручки каждого бренда в общей выручке недели
w14 = brands["rev_w14"] / rev_14
w15 = brands["rev_w15"] / rev_15

m14 = brands["margin_w14"]
m15 = brands["margin_w15"]

# Микс-эффект: вклад каждого бренда за счёт изменения доли выручки
mix_contrib_pp = ((w15 - w14) * m14) * 100

# Внутри-брендовый эффект: вклад за счёт изменения маржинальности бренда
within_contrib_pp = (w15 * (m15 - m14)) * 100

# Общие эффекты
mix_total_pp = mix_contrib_pp.sum()
within_total_pp = within_contrib_pp.sum()
delta_total_pp_check = (m_15 - m_14) * 100

print("Разложение изменения маржинальности:")
print(f"\nМикс-эффект (изменение доли выручки): {mix_total_pp:+.2f} п.п.")
print(f"Внутри-брендовый эффект (изменение маржи брендов): {within_total_pp:+.2f} п.п.")
print(f"Реальное изменение маржинальности: {delta_total_pp_check:+.2f} п.п.")

In [ ]:
# Таблица вкладов в маржинальность
drivers = pd.DataFrame({
    "mix_contrib_pp": mix_contrib_pp,
    "within_contrib_pp": within_contrib_pp,
    "total_contrib_pp": mix_contrib_pp + within_contrib_pp,
}).sort_values("total_contrib_pp", ascending=False)

print("Вклады брендов в изменение маржинальности:")
print(drivers.round(3).to_string())

# Определяем ключевые драйверы маожинальности
top_drivers_pos = drivers.head(3)
top_drivers_neg = drivers.tail(3)

print("\nТоп 3 улучшающих маржинальность:")
print(top_drivers_pos.round(3).to_string())

print("\nТоп 3 ухудшающих маржинальность:")
print(top_drivers_neg.round(3).to_string())

### Аналитическая записка

In [ ]:
# А
top_gp_up = brands.head(3)
top_gp_down = brands.tail(3)
top_m_down = brands.sort_values("delta_margin_pp").head(3)

print("="*80)
print("АНАЛИТИЧЕСКАЯ ЗАПИСКА")
print("Анализ продаж: неделя 15 vs неделя 14")
print("="*80)

print(f"\nИтоговая метрика (TL;DR):")
print(f"\nВыручка изменилась на {delta_rev/rev_14*100:+.2f}% ({delta_rev:,.0f} руб.), валовая прибыль на {delta_gp/gp_14*100:+.2f}% ({delta_gp:+,.0f} руб.)")
print(f"Маржинальность изменилась с {m_14*100:.2f}% до {m_15*100:.2f}% ({delta_m_pp:+.2f} п.п.)")

print(f"\nКлючевые драйверы:")
print(f"Улучшение маржинальности на {delta_m_pp:.2f} п.п. обеспечено:")
print(f"  • Внутри-брендовым эффектом: {within_total_pp:+.2f} п.п. (сами бренды стали маржинальнее)")
print(f"  • Микс-эффектом: {mix_total_pp:+.2f} п.п. (сместилась структура продаж)")

print(f"\nТоп-3 бренда по росту валовой прибыли:")
for idx, (brand_name, row) in enumerate(top_gp_up.iterrows(), 1):
    print(f"  {idx}. {brand_name}: GP {row['delta_gp']:+,.0f} руб., маржа {row['margin_w14']:.1%}→{row['margin_w15']:.1%}")

print(f"\nТоп-3 бренда по падению валовой прибыли:")
for idx, (brand_name, row) in enumerate(top_gp_down.iterrows(), 1):
    print(f"  {idx}. {brand_name}: GP {row['delta_gp']:+,.0f} руб., маржа {row['margin_w14']:.1%}→{row['margin_w15']:.1%}")

print(f"\nПроблемные бренды (падение маржи):")
for idx, (brand_name, row) in enumerate(top_m_down.iterrows(), 1):
    print(f"  {idx}. {brand_name}: маржа {row['margin_w14']:.1%}→{row['margin_w15']:.1%} ({row['delta_margin_pp']:+.2f} п.п.)")

print(f"\nРекомендации (быстрый результат):")
print(f"1. Ограничить скидки/промо в брендах с падением маржи (особенно {top_m_down.index[0]})")
print(f"   Возможный возврат GP: {abs(top_m_down.iloc[0]['delta_margin_pp'])*rev_15/100:,.0f} руб. за неделю при восстановлении маржи")
print(f"2. Усилить продвижение маржинальных лидеров ({top_gp_up.index[0]}, {top_gp_up.index[1]})")
print(f"   Они дают максимальный вклад в прибыль")

print("\n" + "="*80)